# Лабораторная работа: Геометрические алгоритмы в 2D


## Часть 1: Попадание точки в круг и треугольник

### Задание: Проверка геометрических положений

Реализуйте классы **Point**, **Circle** и **Triangle** для проверки попадания точки в круг и треугольник, аналогично лекции 9.

**Класс Point** хранит координаты `x, y` с методом `__str__`.

**Класс Circle** содержит центр (`Point`) и радиус, с методом для проверки `point_in_circle(point)`.

**Класс Triangle** содержит три вершины (`Point`), с методом `point_in_triangle(point)` на основе векторного произведения (barycentric coordinates).

**Генератор случайных точек** внутри ограничивающего прямоугольника, содержащего фигуры:

Создайте генератор `generate_points(xmin, xmax, ymin, ymax, n)`, который случайно создаёт `n` точек с координатами в данном диапазоне.

**Проверку** для всех сгенерированных точек попадания в круг и треугольник.

**Визуализацию** с помощью matplotlib:

- Отобразите круг и треугольник.
- Отметьте **зелёным** цветом точки, попавшие внутрь фигуры, **красным** — вне.
- Используйте различные маркеры для точек в круге и в треугольнике для наглядности.


In [ ]:
import math
import random
import matplotlib.pyplot as plt
from typing import List

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"Point[x={self.x}, y={self.y}]"

In [ ]:
class Circle:
    def __init__(self, point_center: Point, radius):
        self.point_center = point_center
        self.radius = radius

    def __str__(self):
        return f"Circle[center = {self.point_center}, radius = {self.radius}]"

    def point_in_circle(self, point):
        dx = point.x - self.point_center.x
        dy = point.y - self.point_center.y
        return dx**2 + dy**2 <= self.radius ** 2

In [ ]:
class Vector:
    def __init__(self, start_point: Point, end_point: Point):
        self.start_point = start_point
        self.end_point = end_point
        self.x_component = end_point.x - start_point.x
        self.y_component = end_point.y - start_point.y
    
    def __str__(self):
        return f"Vector [start = {self.start_point}, end = {self.end_point}]"

def vector_cross_product(v1, v2):
    return v1.x_component * v2.y_component - v2.x_component * v1.y_component


In [ ]:
class Triangle:
    def __init__(self, v1: Point, v2: Point, v3: Point):
        self.vertex_1 = v1
        self.vertex_2 = v2
        self.vertex_3 = v3

    def triangle_area(self, a, b, c):
        return 0.5*abs(vector_cross_product(Vector(a, b), Vector(a, c)))

    def point_in_triangle(self, point):
        v1, v2, v3 = self.vertex_1, self.vertex_2, self.vertex_3

        S = self.triangle_area(v1, v2, v3)
        if S == 0: return False

        S1 = self.triangle_area(point, v2, v3)
        S2 = self.triangle_area(v1, point, v3)
        S3 = self.triangle_area(v1, v2, point)

        alpha = S1 / S
        beta  = S2 / S
        gamma = S3 / S

        return (
            alpha >= 0 and
            beta >= 0 and
            gamma >= 0 and
            abs(alpha + beta + gamma - 1) < 1e-9
        )

In [ ]:
def generate_points(xmin, xmax, ymin, ymax, n):
    for _ in range(n):
        yield Point(random.uniform(xmin, xmax), random.uniform(ymin, ymax))

In [ ]:
circle = Circle(Point(3, 3), 2)

triangle = Triangle(
    Point(2, 4),
    Point(9, 4),
    Point(7, 8)
)

xmin, xmax = 0, 10
ymin, ymax = 0, 9
points = list(generate_points(xmin, xmax, ymin, ymax, 300))

fig, ax = plt.subplots(figsize=(8, 8))

circle_plot = plt.Circle((circle.point_center.x, circle.point_center.y), circle.radius, fill=False, color='blue')
ax.add_patch(circle_plot)

tx = [triangle.vertex_1.x, triangle.vertex_2.x, triangle.vertex_3.x, triangle.vertex_1.x]
ty = [triangle.vertex_1.y, triangle.vertex_2.y, triangle.vertex_3.y, triangle.vertex_1.y]
ax.plot(tx, ty, color="purple")

for p in points:
    inside_c = circle.point_in_circle(p)
    inside_t = triangle.point_in_triangle(p)

    if inside_c and inside_t:
        ax.scatter(p.x, p.y, color="green", s=18, marker="s")
    elif inside_c:
        ax.scatter(p.x, p.y, color="green", s=18, marker="o")
    elif inside_t:
        ax.scatter(p.x, p.y, color="green", s=18, marker="^")
    else:
        ax.scatter(p.x, p.y, color="red", s=15)

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
plt.title("Попадание точек в круг и треугольник")
plt.show()


## Часть 2: Пересечение отрезков и окружностей

### Задание: Алгоритмы геометрического пересечения

Расширьте систему классов из Части 1 для проверки пересечения отрезков с окружностями и отрезков между собой.

**Класс Segment** содержит две вершины (`Point` A и B), с методами:
- `__str__` для отображения отрезка
- `length()` — вычисление длины отрезка
- `intersects_circle(circle)` — проверка пересечения отрезка с окружностью (возвращает список точек пересечения или пустой список)
- `intersects_segment(other)` — проверка пересечения двух отрезков (возвращает точку пересечения или None)

**Реализация алгоритмов пересечения:**
- **Отрезок ↔ Окружность**: Решение квадратного уравнения для расстояния от центра окружности до отрезка 
- **Отрезок ↔ Отрезок**: Метод на основе векторного произведения и параметризации (ориентация точек) 

**Генератор тестовых конфигураций:**
- `generate_segments(n, xmin, xmax, ymin, ymax)` — генератор n случайных отрезков в прямоугольнике
- Создайте несколько тестовых окружностей и отобразите все возможные случаи пересечения

**Визуализация с помощью matplotlib:**
- Отобразите окружность(и) и множество отрезков
- **Зелёные отрезки** — пересекают окружность (отметьте точки пересечения)
- **Синие отрезки** — полностью внутри окружности
- **Красные отрезки** — полностью вне окружности  
- **Жёлтые точки** — точки пересечения отрезков между собой
- Используйте легенду и сетку для наглядности


In [ ]:
class Segment:
    def __init__(self, start_point, end_point):
        self.start_point = start_point
        self.end_point = end_point

    def __str__(self):
        return f"Segment[start = {self.start_point}, end = {self.end_point}]"

    def length(self):
        dx = self.end_point.x - self.start_point.x
        dy = self.end_point.y - self.start_point.y
        return (dx**2 + dy**2)**0.5

    def intersects_circle(self, circle: Circle):
        Ax, Ay = self.start_point.x, self.start_point.y
        Bx, By = self.end_point.x, self.end_point.y
        Cx, Cy = circle.point_center.x, circle.point_center.y
        r = circle.radius

        dx = Bx - Ax
        dy = By - Ay
        fx = Ax - Cx
        fy = Ay - Cy

        a = dx**2 + dy**2
        b = 2*(fx*dx + fy*dy)
        c = fx**2 + fy**2 - r**2

        D = b**2 - 4*a*c
        if D < 0:
            return []

        result = []
        sqrtD = math.sqrt(D)
        t1 = (-b - sqrtD) / (2*a)
        t2 = (-b + sqrtD) / (2*a)

        for t in (t1, t2):
            if 0 <= t <= 1:
                x = Ax + t*dx
                y = Ay + t*dy
                result.append(Point(x, y))
        return result

    def intersects_segment(self, other):
        A, B = self.start_point, self.end_point
        C, D = other.start_point, other.end_point

        def range_intersection(s1, e1, s2, e2):
            if s1 > e1: s1, e1 = e1, s1
            if s2 > e2: s2, e2 = e2, s2
            return max(s1, s2) <= min(e1, e2)

        if not (range_intersection(A.x, B.x, C.x, D.x) and
                range_intersection(A.y, B.y, C.y, D.y)):
            return None

        AB = Vector(A, B)
        AC = Vector(A, C)
        AD = Vector(A, D)
        CD = Vector(C, D)
        CA = Vector(C, A)
        CB = Vector(C, B)

        d1 = vector_cross_product(AB, AC)
        d2 = vector_cross_product(AB, AD)
        d3 = vector_cross_product(CD, CA)
        d4 = vector_cross_product(CD, CB)

        if ((d1 >= 0 and d2 <= 0) or (d1 <= 0 and d2 >= 0)) and ((d3 >= 0 and d4 <= 0) or (d3 <= 0 and d4 >= 0)):
            denom = vector_cross_product(AB, CD)
            if denom == 0:
                return None

            px = ((A.x*B.y - A.y*B.x)*(C.x - D.x) - (A.x - B.x)*(C.x*D.y - C.y*D.x)) / denom
            py = ((A.x*B.y - A.y*B.x)*(C.y - D.y) - (A.y - B.y)*(C.x*D.y - C.y*D.x)) / denom
            return Point(px, py)
        return None

In [ ]:

def generate_segments(n, xmin, xmax, ymin, ymax):
    for _ in range(n):
        A = Point(random.uniform(xmin, xmax), random.uniform(ymin, ymax))
        B = Point(random.uniform(xmin, xmax), random.uniform(ymin, ymax))
        yield Segment(A, B)

circle1 = Circle(Point(5, 5), 3)
segments = list(generate_segments(25, 0, 10, 0, 10))

intersection_points = []

fig, ax = plt.subplots(figsize=(8, 8))

used_labels = {
    "green": False,
    "blue": False,
    "red": False,
    "point": False,
    "seg_point": False
}

ax.add_patch(plt.Circle((circle1.point_center.x, circle1.point_center.y), circle1.radius, fill=False, color="black", linewidth=2))

for i, seg in enumerate(segments):
    points_on_circle = seg.intersects_circle(circle1)

    dx_start = seg.start_point.x - circle1.point_center.x
    dy_start = seg.start_point.y - circle1.point_center.y
    dist_start = (dx_start**2 + dy_start**2)**0.5

    dx_end = seg.end_point.x - circle1.point_center.x
    dy_end = seg.end_point.y - circle1.point_center.y
    dist_end = (dx_end**2 + dy_end**2)**0.5

    if len(points_on_circle) > 0:
        color = "green"
    elif dist_start < circle1.radius and dist_end < circle1.radius:
        color = "blue"
    else:
        color = "red"

    label_map = {
        "green": "Пересекает окружность",
        "blue": "Отрезок внутри окружности",
        "red": "Отрезок вне окружности"
    }

    label = label_map[color] if not used_labels[color] else None
    used_labels[color] = True

    ax.plot([seg.start_point.x, seg.end_point.x], [seg.start_point.y, seg.end_point.y], color=color, linewidth=2, label=label)
    for p in points_on_circle:
        label = "Точка пересечения окружности и отрезка" if not used_labels["point"] else None
        used_labels["point"] = True

        ax.scatter(p.x, p.y, color="yellow", edgecolor="black", s=60, label=label, zorder=2)

for i in range(len(segments)):
    for j in range(i + 1, len(segments)):
        p = segments[i].intersects_segment(segments[j])
        if p:
            intersection_points.append(p)

for p in intersection_points:
    label = "Точка пересечения отрезков" if not used_labels["seg_point"] else None
    used_labels["seg_point"] = True
    ax.scatter(p.x, p.y, color="purple", edgecolor="black", s=60, label=label, zorder=2)

ax.grid(True, alpha=0.3)
ax.set_aspect("equal")
plt.title("Пересечение отрезков и окружностей")
ax.legend()
plt.show()


## Часть 3: Формула Гаусса для площади многоугольника

### Задание: Вычисление площади произвольных многоугольников

Реализуйте класс **Polygon** для работы с многоугольниками и вычисления их площади по формуле Гаусса (шнуровки).

**Класс Polygon** содержит список вершин (`List[Point]`), с методами:
- `__init__(vertices: List[Point])` — конструктор (проверка на самопересечение опционально)
- `__str__()` — отображение координат вершин
- `area()` — вычисление площади по формуле Гаусса:  
  $S = \frac{1}{2} \left| \sum_{i=1}^{n} (x_i y_{i+1} - x_{i+1} y_i) \right| $,  
  где по соглашению \( (x_{n+1}, y_{n+1}) = (x_1, y_1) \).
- `is_convex()` — проверка выпуклости многоугольника
- `centroid()` — вычисление центра масс (средневзвешенных координат)




**Генератор многоугольников:**
- `generate_polygons(n, xmin, xmax, ymin, ymax, min_sides=3, max_sides=8)` — генератор n случайных выпуклых/вогнутых многоугольников
- `generate_regular_polygon(sides, center, radius)` — правильный многоугольник

**Визуализация с помощью matplotlib:**
- Отобразите несколько многоугольников разных форм
- **Заливка** пропорциональна площади (alpha=0.5, разные цвета)
- **Подписи** с точной площадью рядом с каждым многоугольником
- **Сравнение** с площадью по разбиению на треугольники (для проверки)
- **Легенда** с площадями и типом (выпуклый/вогнутый)


In [ ]:
class Polygon:
    def __init__(self, vertices: List[Point]):
        self.vertices = vertices

    def __str__(self):
        return "Polygon[" + ", ".join(str(v) for v in self.vertices) + "]"

    def area(self):
        s = 0
        n = len(self.vertices)
        for i in range(n):
            x1, y1 = self.vertices[i].x, self.vertices[i].y
            x2, y2 = self.vertices[(i + 1) % n].x, self.vertices[(i + 1) % n].y
            s += x1 * y2 - x2 * y1
        return abs(s) / 2

    def is_convex(self):
        signs = []
        n = len(self.vertices)

        for i in range(n):
            A = self.vertices[i]
            B = self.vertices[(i + 1) % n]
            C = self.vertices[(i + 2) % n]

            v1 = Vector(A, B)
            v2 = Vector(B, C)

            cross = vector_cross_product(v1, v2)

            if cross != 0:
                signs.append(cross > 0)
        return all(signs) or not any(signs)

    def centroid(self):
        A = self.area()
        if A == 0:
            return None

        cx = 0
        cy = 0
        n = len(self.vertices)

        for i in range(n):
            x1, y1 = self.vertices[i].x, self.vertices[i].y
            x2, y2 = self.vertices[(i + 1) % n].x, self.vertices[(i + 1) % n].y
            cross = x1 * y2 - x2 * y1
            cx += (x1 + x2) * cross
            cy += (y1 + y2) * cross

        cx /= (6 * A)
        cy /= (6 * A)

        return Point(cx, cy)


In [ ]:
def generate_regular_polygon(sides, center: Point, radius):
    vertices = []
    cx, cy = center.x, center.y

    for i in range(sides):
        angle = 2 * math.pi * i / sides
        x = cx + radius * math.cos(angle)
        y = cy + radius * math.sin(angle)
        vertices.append(Point(x, y))

    return Polygon(vertices)


In [ ]:
def generate_polygons(n, xmin, xmax, ymin, ymax, min_sides=3, max_sides=8):
    for _ in range(n):
        sides = random.randint(min_sides, max_sides)

        angles = sorted(random.random() * 2 * math.pi for _ in range(sides))

        cx = random.uniform(xmin + 1, xmax - 1)
        cy = random.uniform(ymin + 1, ymax - 1)

        vertices = []
        for angle in angles:
            r = random.uniform(0.5, 3)
            x = cx + r * math.cos(angle)
            y = cy + r * math.sin(angle)
            vertices.append(Point(x, y))

        yield Polygon(vertices)


In [ ]:
def triangulated_area(poly: Polygon):
    area_sum = 0
    A = poly.vertices[0]

    for i in range(1, len(poly.vertices) - 1):
        B = poly.vertices[i]
        C = poly.vertices[i + 1]

        v1 = Vector(A, B)
        v2 = Vector(A, C)

        area_sum += vector_cross_product(v1, v2) / 2

    return area_sum


In [ ]:
polygons = list(generate_polygons(6, 0, 10, 0, 10))

regular_poly = generate_regular_polygon(sides=6, center=Point(5, 5), radius=2)
polygons.append(regular_poly)

fig, ax = plt.subplots(figsize=(10, 10))

colors = ['red', 'green', 'blue', 'purple', 'orange', 'cyan','magenta']

for i, poly in enumerate(polygons):
    xs = [p.x for p in poly.vertices] + [poly.vertices[0].x]
    ys = [p.y for p in poly.vertices] + [poly.vertices[0].y]

    area = poly.area()
    tri_area = triangulated_area(poly)
    centroid = poly.centroid()

    color = colors[i % len(colors)]

    ax.fill(xs, ys, color=color, alpha=0.35)
    ax.plot(xs, ys, color=color, linewidth=2)

    if centroid:
        ax.scatter(centroid.x, centroid.y, color='black', s=40)

    ax.text(
        centroid.x + 0.1, centroid.y + 0.1,
        f"S={area:.2f}\nTri={tri_area:.2f}\n{'Выпуклый' if poly.is_convex() else 'Вогнутый'}",
        fontsize=10, weight='bold'
    )

ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
plt.title("Многоугольники: Формула Гаусса, выпуклость, центр масс")
plt.show()


# Прикладная задача
## Оценка площади озера по спутниковому снимку 

По результатам сегментации спутникового изображения береговая линия озера аппроксимирована многоугольником с вершинами в пиксельных координатах. Известно пространственное разрешение снимка (например, 1 пиксель = 2 м по обеим осям).

### Задача
- Перевести координаты вершин из пикселей в метры, умножив каждую координату на масштаб:  
  $x'_i = s_x * x_i,  y'_i = s_y * y_i$  (для квадратного пикселя s_x = s_y = s).
- С помощью формулы Гаусса вычислить площадь озера в квадратных метрах и гектарах:  
  $S = 1/2 * | Σ_{i=1}^{n} (x'_i y'_{i+1} - x'_{i+1} y'_i) |, где (x'_{n+1}, y'_{n+1}) = (x'_1, y'_1)$; затем S_га = S / 10 000.
- Сравнить полученную площадь с заданным порогом (например, 10 гектаров), чтобы классифицировать объект как «малое озеро» или «крупный водоём».


## Чтобы узнать вариант введите ваш номер в списке

In [ ]:
print(int(input("Введите номер в списке: "))%6)

## Вариант 1: Озеро в Карелии (малый водоём)

**Название объекта:** Озеро Малое Круглое  
**Местоположение:** Республика Карелия  

**Координаты береговой линии (пиксели на снимке):**

[
(145, 87), (162, 76), (188, 81), (201, 94),
(208, 112), (194, 128), (172, 135), (151, 130),
(133, 117), (126, 101), (129, 89)
]

**Пространственное разрешение:** 1 пиксель = 1.5 метра  
**Порог классификации:** 5 гектаров (для малых озёр)  

---
## Вариант 2: Старица реки Волги

**Название объекта:** Старица "Подкова"  
**Местоположение:** Волго-Ахтубинская пойма, Астраханская область  

**Координаты (пиксели):**

[
(320, 180), (340, 165), (370, 160), (400, 165),
(420, 180), (435, 200), (440, 225), (430, 250),
(410, 265), (380, 270), (350, 260), (330, 240),
(315, 220), (310, 200)
]

**Пространственное разрешение:** 1 пиксель = 2.0 метра  
**Порог классификации:** 8 гектаров  

---

## Вариант 3: Городской пруд (искусственный водоём)

**Название объекта:** Парковый пруд "Зеркальный"  
**Местоположение:** Городской парк, Москва  

**Координаты (пиксели):**

[
(80, 120), (110, 100), (150, 95), (190, 105),
(220, 125), (235, 155), (220, 185), (190, 205),
(150, 210), (110, 200), (80, 180), (65, 150)
]

**Пространственное разрешение:** 1 пиксель = 0.5 метра (высокое разрешение)  
**Порог классификации:** 2 гектара (для городских объектов)  

---

## Вариант 4: Альпийское ледниковое озеро

**Название объекта:** Ледниковое озеро "Голубое"  
**Местоположение:** Кавказские горы, высота 2800 м  

**Координаты (пиксели):**

[
(50, 70), (85, 55), (125, 50), (165, 60),
(195, 80), (210, 110), (200, 145), (170, 170),
(130, 180), (90, 170), (60, 150), (40, 120),
(35, 95)
]

**Пространственное разрешение:** 1 пиксель = 4.0 метра (низкое разрешение горной съёмки)  
**Порог классификации:** 15 гектаров (горные озёра)  

---

## Вариант 5: Водохранилище на малой реке

**Название объекта:** Водохранилище "Приозерное"  
**Местоположение:** Смоленская область  

**Координаты (пиксели):**

[
(200, 100), (250, 80), (310, 85), (360, 105),
(400, 140), (420, 185), (405, 230), (360, 260),
(300, 270), (240, 250), (200, 220), (180, 180),
(175, 140), (185, 115)
]

**Пространственное разрешение:** 1 пиксель = 3.0 метра  
**Порог классификации:** 20 гектаров (водохранилища)  

---

## Вариант 6: Техногенный карьер, заполненный водой

**Название объекта:** Затопленный карьер "Северный"  
**Местоположение:** Кемеровская область, бывший угольный разрез  

**Координаты (пиксели):**

[
(90, 60), (130, 40), (180, 35), (230, 45),
(270, 70), (295, 105), (300, 145), (285, 185),
(250, 215), (200, 225), (150, 210), (110, 180),
(85, 140), (80, 100)
]

**Пространственное разрешение:** 1 пиксель = 2.5 метра  
**Порог классификации:** 10 гектаров (техногенные объекты)  


In [ ]:
pixel_vertices = [(80, 120), (110, 100), (150, 95), (190, 105),
    (220, 125), (235, 155), (220, 185), (190, 205),
    (150, 210), (110, 200), (80, 180), (65, 150)]

scale = 0.5
threshold_hectares = 2

vertices = [Point(x * scale, y * scale) for x, y in pixel_vertices]
polygon = Polygon(vertices)

area_m2 = polygon.area()
area_hectares = area_m2 / 10000

if area_hectares >= threshold_hectares:
    category = "крупный водоём"
else:
    category = "малое озеро"

print("Площадь:", area_m2, "м²")
print("Площадь:", area_hectares, "га")
print("Категория:", category)
